# Memoria

**Sumario**

1. Memoria conversacional de búfer
   1. `ConversationBufferMemory`
   2. `ConversationBufferWindowMemory`
   3. `ConversationTokenBufferMemory`
   4. `ConversationSummaryMemory`
<br></br>
1. Memoria conversacional con entidades
   1. `ConversationEntityMemory`
<br></br>
1. Memoria conversacional con un grafo de conocimiento
   1. `ConversationKGMemory`
<br></br>
1. Memoria basada en bases de datos vectoriales
   1. `VectorStoreRetrieverMemory`

In [ ]:
import openai
import os
from langchain.llms import AzureOpenAI

# Lee la clave de API desde el archivo de configuración
with open('config.txt') as f:
    config = dict(line.strip().split('=') for line in f)

openai.api_type = "azure"
openai.api_base = "https://gpt3tests.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = config.get("OPENAI_API_KEY", "")

# Nombre del despliegue en mi Azure OpenAI Studio is "Davinci003", el modelo es "text-davinci-003"
engine = "Davinci003"
model = "text-davinci-003"
openai_api_version = "2023-12-01" 

# Nombre del despliegue en mi Azure OpenAI Studio para el modelo de embeddings es "TextEmbeddingAda002"
embeddings_engine = "TextEmbeddingAda002"

max_tokens = 1000

llm = AzureOpenAI(
    azure_endpoint=openai.api_base, 
    azure_deployment=engine, 
    openai_api_key=config.get("OPENAI_API_KEY", ""), 
    openai_api_version=openai.api_version,
    # temperature=0, # Podemos poner la temperatura a 0 si queremos reducir la variabilidad de las respuestas
)
llm.openai_api_base = openai.api_base 
llm.max_tokens = max_tokens

# 1 - Memoria conversacional de búfer

La memoria de búfer es la forma más simple de memoria. Simplemente implica mantener un búfer de todos los mensajes anteriores. Hay diferentes tipos:

* `ConversationBufferMemory`. 
* `ConversationBufferWindowMemory`. 
* `ConversationTokenBufferMemory`. 
* `ConversationSummaryMemory`. 


## 1.1 - `ConversationBufferMemory`

Permite almacenar mensajes y luego extraer los mensajes en una variable

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

template = """
Lo siguiente es una conversación amistosa entre un humano y una inteligencia artificial. La IA es conversadora y proporciona muchos detalles específicos de su contexto. Si la IA no conoce la respuesta a una pregunta, dice honestamente que no lo sabe.

Conversación actual:
{history}
Humano: {input}
IA:
"""

chat_prompt = PromptTemplate.from_template(template)
buffer_memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    prompt=chat_prompt,
    memory=buffer_memory,
    verbose=True, 
)

### Actualicemos el prompt de entrada al español

Antes de empezar, si nos fijamos, la cadena tiene una prompt en inglés que si bien no deberia impedir que el modelo funcionase correctamente porque la tarea es muy simple, es conveniente que la actualicemos, ya que vamos a conversar en español.

In [ ]:
conversation.prompt

In [ ]:
conversation.prompt = chat_prompt

conversation.prompt

In [ ]:
conversation.predict(input="Hola, me llamo Fernando")

In [ ]:
conversation.predict(input="¿Cuanto es 1+1?")

In [ ]:
conversation.predict(input="¿Cómo me llamo?")

### Como acceder al búfer de memoria

In [ ]:
buffer_memory.save_context({"input": "¿Cual es el modelo que te soporta?"}, 
                    {"output": "Mi modelo es GPT-3"})

print(buffer_memory.buffer)

In [ ]:
conversation.predict(input="¿Cual decias que era tu modelo?")

## 1.2 - `ConversationBufferWindowMemory`

Mantiene una lista de las interacciones de la conversación a lo largo del tiempo. Solo utiliza las últimas $K$ interacciones.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

buffer_window_memory = ConversationBufferWindowMemory(k=1) # Solo va a recordar 1 interacción que hayamos hecho
conversation = ConversationChain(
    llm=llm, 
    prompt=chat_prompt,
    memory=buffer_window_memory,
    verbose=True, 
)

In [ ]:
conversation.predict(input="Hola, me llamo Fernando")

In [ ]:
conversation.predict(input="¿Cuanto es 1+1?")

In [ ]:
conversation.predict(input="¿Cómo me llamo?")

## 1.3 - `ConversationTokenBufferMemory`

Mantiene un búfer de interacciones recientes en la memoria y utiliza la longitud de tokens en lugar del número de interacciones para determinar cuándo "cortar"

In [ ]:
from langchain.memory import ConversationTokenBufferMemory

buffer_token_memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=20) # Guarda los últimos X tokens
buffer_token_memory.save_context({"input": "¿Cómo se llama el planeta en el que vivimos"},
                    {"output": "La Tierra"})
buffer_token_memory.save_context({"input": "¿En qué galaxia se situa?"},
                    {"output": "La Via Láctea"})
buffer_token_memory.save_context({"input": "¿En qué sistema?"}, 
                    {"output": "En el Sistema Solar"})

Podemos ver como cambia el contexto segun cambiamos el número máximo de tokens permitidos:

In [ ]:
buffer_token_memory.load_memory_variables({})

## 1.4 - `ConversationSummaryMemory`

En lugar de limitar el número de intercambios o tokens, podemos escribir un resumen de la conversación hasta ahora y lo usamos como memoria. 

Este enfoque mantiene la última interacción en la memoria y resume las anteriores. Así que, cuando llega una nueva interacción, la anterior se combina con el resumen.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

agenda = """
- Hay una reunion a las 10 am con el equipo de ingenieria, asi que necesitas tener preparada la presentación PowerPoint.
- De 11 am a 13 pm hay que trabajar en el proyecto de Python, lo cual será rápido porque el proyecto esta casi terminado.
- Al mediodía, almuerzo en el restaurante italiano con un cliente ha conducido más de una hora para poder reunirse contigo. Asegurate de llevar el portátil para mostrar la última demo del modelo LLM que has preparado
"""

buffer_summary_memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
buffer_summary_memory.save_context({"input": "Hola"}, {"output": "¿Qué tal?"})
buffer_summary_memory.save_context({"input": "No mucho, solo pasando el rato"},
                     {"output": "Genial"})
buffer_summary_memory.save_context({"input": "¿Qué hay en la agenda hoy?"}, 
                     {"output": f"{agenda}"})

buffer_summary_memory.load_memory_variables({})

Langchain tiene varios prompts por defecto escritos en inglés que pueden modificar la memoria o la salida del modelo. Por ello, en casos como este, podemos actualizar el prompt para que se adecue más a nuestras necesidades:

In [ ]:
from langchain.memory.prompt import SUMMARY_PROMPT

SUMMARY_PROMPT

In [ ]:
summary_template = """
Resume progresivamente las líneas de conversación proporcionadas, añadiendo al resumen anterior para devolver un nuevo resumen.

EJEMPLO
Resumen actual:
El humano pregunta qué piensa la IA sobre la inteligencia artificial. La IA piensa que la inteligencia artificial es una fuerza para el bien.

Nuevas líneas de conversación:
Humano: ¿Por qué piensas que la inteligencia artificial es una fuerza para el bien?
IA: Porque la inteligencia artificial ayudará a los humanos a alcanzar su máximo potencial.

Nuevo resumen:
El humano pregunta qué piensa la IA sobre la inteligencia artificial. La IA piensa que la inteligencia artificial es una fuerza para 
el bien porque ayudará a los humanos a alcanzar su máximo potencial.
FIN DEL EJEMPLO

Resumen actual:
{summary}

Nuevas líneas de conversación:
{new_lines}

Nuevo resumen:
"""
summary_prompt = PromptTemplate.from_template(summary_template)

buffer_summary_memory = ConversationSummaryBufferMemory(
    llm=llm, 
    max_token_limit=100,
    prompt=summary_prompt,
)
buffer_summary_memory.save_context({"input": "Hola"}, {"output": "¿Qué tal?"})
buffer_summary_memory.save_context({"input": "No mucho, solo pasando el rato"},
                     {"output": "Genial"})
buffer_summary_memory.save_context({"input": "¿Qué hay en la agenda hoy?"}, 
                     {"output": f"{agenda}"})

buffer_summary_memory.load_memory_variables({})

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory=buffer_summary_memory,
    verbose=True, 
    prompt=chat_prompt,
)

conversation.predict(input="¿Tengo alguna reunion a las 10am?")

In [ ]:
conversation.predict(input="¿Cuantas diapositivas aconsejas para mi presentación?")

In [ ]:
buffer_summary_memory.load_memory_variables({})

# 2 - Memoria conversacional con entidades

## 2.1 - `ConversationEntityMemory`

Esta memoria extrae información sobre las entidades (utilizando LLMs) y construye su conocimiento sobre cada entidad según avanza la conversación

Hemos preparado un ejemplo con 5 entidades (conocidas de antemano por el modelo y desconocidas tambié):
* Jorge (desconocido)
* Ignacio (desconocido)
* Langchain (desconocido en 2021, es decir, datos del último modelo)
* Nielsen (conocido)
* Everest (conocido)

Al generar el resumen de las entidades, podemos ver cómo el modelo incorpora información adicional a las entidades "conocidas" dada la información contextual proporcionada.

In [ ]:
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE

# Mostramos el template de base (en inglés)
print(ENTITY_MEMORY_CONVERSATION_TEMPLATE.template)

Nuestro primer paso como en casos anteriores es modificar adecuadamente el prompt para que se encuentre en español:

In [ ]:
entity_prompt_template = """
Eres un asistente para un humano, alimentado por un modelo de lenguaje grande entrenado por OpenAI.

Estás diseñado para poder ayudar con una amplia gama de tareas, desde responder preguntas simples hasta proporcionar explicaciones detalladas y participar en discusiones sobre una amplia variedad de temas. Como modelo de lenguaje, puedes generar texto similar al humano basado en la entrada que recibes, lo que te permite participar en conversaciones con un tono natural y proporcionar respuestas coherentes y relevantes al tema en cuestión.

Estás constantemente aprendiendo y mejorando, y tus capacidades están en constante evolución. Puedes procesar y entender grandes cantidades de texto y utilizar este conocimiento para ofrecer respuestas precisas e informativas a una amplia variedad de preguntas. Tienes acceso a información personalizada proporcionada por el humano en la sección de Contexto a continuación. Además, puedes generar tu propio texto basado en la entrada que recibes, lo que te permite participar en discusiones y proporcionar explicaciones y descripciones sobre una amplia variedad de temas.

En general, eres una herramienta poderosa que puede ayudar con una amplia gama de tareas y proporcionar información valiosa sobre una amplia variedad de temas. Ya sea que el humano necesite ayuda con una pregunta específica o simplemente quiera tener una conversación sobre un tema en particular, estás aquí para ayudar.

Contexto:
{entities}

Conversación actual:
{history}
Última línea:
Humano: {input}
Tú:
"""

entity_prompt = PromptTemplate.from_template(entity_prompt_template)

In [ ]:
from langchain.memory import ConversationEntityMemory

entity_memory = ConversationEntityMemory(llm=llm)
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    prompt=entity_prompt,
    memory=entity_memory
)

In [ ]:
conversation.predict(input="Jorge e Ignacio estan trabajando en un proyecto para Nielsen. Jorge es economista e Ignacio es ingeniero")

In [ ]:
conversation.memory.entity_store.store

In [ ]:
conversation.predict(input="Que tipo de empresa es Nielsen?")

In [ ]:
conversation.predict(input="Jorge tiene el sueño de escalar el monte Everest")

In [ ]:
conversation.memory.entity_store.store

# 3 - Memoria basada con un grafo de conocimiento

## 3.1 - `ConversationKGMemory`

En este caso utilizamos el modelo va generando un pseudo grafo de conocimiento (en formato de texto) donde va almacenando la informacion relevante de la conversación, centrandose sobretodo en las diferentes entidades.

Vamos a utilizar el template que hicimos anteriormente pero sin la variable `entities`, ya que no es utilizada por `ConversationKGMemory`.

In [ ]:
kg_prompt_template = """
Eres un asistente para un humano, alimentado por un modelo de lenguaje grande entrenado por OpenAI.

Estás diseñado para poder ayudar con una amplia gama de tareas, desde responder preguntas simples hasta proporcionar explicaciones detalladas y participar en discusiones sobre una amplia variedad de temas. Como modelo de lenguaje, puedes generar texto similar al humano basado en la entrada que recibes, lo que te permite participar en conversaciones con un tono natural y proporcionar respuestas coherentes y relevantes al tema en cuestión.

Estás constantemente aprendiendo y mejorando, y tus capacidades están en constante evolución. Puedes procesar y entender grandes cantidades de texto y utilizar este conocimiento para ofrecer respuestas precisas e informativas a una amplia variedad de preguntas. Tienes acceso a información personalizada proporcionada por el humano en la sección de Contexto a continuación. Además, puedes generar tu propio texto basado en la entrada que recibes, lo que te permite participar en discusiones y proporcionar explicaciones y descripciones sobre una amplia variedad de temas.

En general, eres una herramienta poderosa que puede ayudar con una amplia gama de tareas y proporcionar información valiosa sobre una amplia variedad de temas. Ya sea que el humano necesite ayuda con una pregunta específica o simplemente quiera tener una conversación sobre un tema en particular, estás aquí para ayudar.

Contexto:

Conversación actual:
{history}
Última línea:
Humano: {input}
Tú:
"""

kg_prompt = PromptTemplate.from_template(kg_prompt_template)

In [ ]:
from langchain.memory import ConversationKGMemory

kg_memory = ConversationKGMemory(llm=llm)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    prompt=kg_prompt,
    memory=kg_memory
)

In [ ]:
conversation.predict(input="Jorge e Ignacio estan trabajando en un proyecto para Nielsen. Jorge es economista e Ignacio es ingeniero")
conversation.predict(input="Que tipo de empresa es Nielsen?")
conversation.predict(input="Jorge tiene el sueño de escalar el monte Everest")

In [ ]:
conversation.predict(input="Que sabes de Ignacio?")

In [ ]:
kg_memory.kg.get_triples()

Esta funcionalidad es similar a la que ofrece `ConversationEntityMemory`, pero organiza la informacion en forma de tripletas de un grafo de conocimiento.

Además de ofrecernos la posibilidad de ver el grafo de conocimiento que el LLM va generando, LangChain nos ofrece metodos para interactuar con el mismo:
* Extraccion de entidades
* Extraccion de tripletas

Pero mi experiencia personal es que no funciona muy bien, ya que no está 100% relacionado con el grafo que tiene en memoria por lo que resulta en un comportamiento extraño. Mi consejo en este caso seria hacer algo adhoc en vez de utilizar estos métodos.

In [ ]:
kg_memory.get_current_entities({"input":"Que sabes de Ignacio, Nielsen y Google?"}) # Extrae entidades de una frase, pero no tiene porque estar relacionado con su KG interno

In [ ]:
kg_memory.get_knowledge_triplets({"input":"Que sabes de Ignacio, Nielsen y Google?"})# Extrae tripletas de una frase, pero no funciona muy bien

# 4 - Memoria basada en bases de datos vectoriales

## 4.1 - `VectorStoreRetrieverMemory`

`VectorStoreRetrieverMemory` almacena recuerdos en un VectorDB y consulta los documentos más "relevantes" principales cada vez que se llama. Esto difiere de la mayoría de las otras clases de memoria en que no sigue explícitamente el orden de las interacciones.

En este caso, los "documentos" son fragmentos anteriores de la conversación. Esto puede ser útil para referirse a piezas relevantes de información que la IA recibió anteriormente en la conversación.

```python
pip install faiss-gpu # For CUDA 7.5+ Supported GPU's.
# OR
pip install faiss-cpu # For CPU Installation
```

In [ ]:
from langchain.embeddings import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=openai.api_base, 
    azure_deployment=embeddings_engine, 
    openai_api_key=config.get("OPENAI_API_KEY", ""), 
    openai_api_version=openai.api_version
)

In [ ]:
text = "Esto es una prueba"

query_result = embeddings.embed_query(text)

print(len(query_result)) # Devuelve un vector de dimension 1536
# query_result

Podemos usar estos embeddings con una Base de datos vectorial como [FAISS](https://github.com/facebookresearch/faiss):

In [ ]:
import faiss

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS
from langchain.memory import VectorStoreRetrieverMemory

embedding_size = 1536 # Dimensions of the OpenAIEmbeddings
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings, index, InMemoryDocstore({}), {})

vector_retriever = vectorstore.as_retriever(search_kwargs=dict(k=2)) # Devolvemos K documentos
vector_memory  = VectorStoreRetrieverMemory(retriever=vector_retriever)

In [ ]:
vector_memory.save_context({"input": "Mi comida favorita es la pizza"}, {"output": "Gran elección!"})
vector_memory.save_context({"input": "Todas las tarde juego al fútbol"}, {"output": "El fútbol es muy entretenido."})
vector_memory.save_context({"input": "No me gusta el golf"}, {"output": "..."})
vector_memory.save_context({"input": "Mi ciudad favorita es Paris"}, {"output": "Genial"})

In [ ]:
print(vector_memory.load_memory_variables({"prompt": "Que deporte me aconsejas para ver en la tele?"})["history"])

Ahora veamos como usarlo con un cadena coversacional. Para ello, vamos a modificar levemente el prompt de tal forma que el modelo utilice los "documentos" recuperados para basar su respuesta si lo considera adecuado

In [ ]:
vector_prompt_template = """
Lo siguiente es una conversación amistosa entre un humano y una inteligencia artificial. La IA es conversadora y proporciona muchos detalles específicos de su contexto. Si la IA no conoce la respuesta a una pregunta, dice honestamente que no lo sabe.

Interacciones previas en la conversación que son relevantes:
{history}

(No necesitas utilizar estas interacciones previas para basar tu respuesta si no lo consideras relevante)

Conversación actual:
Humano: {input}
IA:
"""

vector_prompt = PromptTemplate.from_template(vector_prompt_template)

In [ ]:
conversation = ConversationChain(
    llm=llm,
    prompt=vector_prompt,
    memory=vector_memory,
    verbose=True,
)

In [ ]:
conversation.predict(input="¿Te gusta el deporte?") 

# Siguiente

En la siguiente lección, veremos que es un agente, como utilizarlo, y sus principales usos.